In [16]:
import torch, torchdata, torchtext
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset

In [17]:
torch.__version__, torchdata.__version__, torchtext.__version__

('2.2.2+cu121', '0.7.1', '0.17.2+cpu')

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
SEED = 1234 #change three times
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## 1. load dataset

In [20]:
df = pd.read_csv('data/pre-category/sample500percategoty.csv')
df.head(5)

,questionType,question
0,yes/no,is this compatible with kenmore 2203980?
1,yes/no,Will this work with sears 758 144521
2,yes/no,can i still use dryer until its fixed
3,yes/no,Is this the same size as the A122?
4,yes/no,does it have a turntable?


In [21]:
# convert to lower case
df['question']  =  df['question'].apply(lambda x: x.lower() if isinstance(x, str) else x)

In [22]:
# tranform the text question type to integer
df['questionType']=df['questionType'].replace(['yes/no','open-ended'], [0,1])
df.head(5)

,questionType,question
0,0,is this compatible with kenmore 2203980?
1,0,will this work with sears 758 144521
2,0,can i still use dryer until its fixed
3,0,is this the same size as the a122?
4,0,does it have a turntable?


In [23]:
def data_cleaning(data):
    regex_s = re.sub("\\(.+?\\)|[\r\n|\n\r]|!", "", data)
    fin = " ".join(regex_s.split())
    return fin

In [24]:
df['question'] = df['question'].apply(data_cleaning)

In [25]:
train_df, val_df = train_test_split(df, test_size=0.15,stratify=df['questionType'], random_state=SEED)

In [26]:
train_df, test_df = train_test_split(train_df, test_size=0.15, stratify=train_df['questionType'],random_state=SEED)

In [27]:
train_df['questionType'].value_counts()

questionType
1    3793
0    3793
Name: count, dtype: int64

In [28]:
val_df['questionType'].value_counts()

questionType
1    788
0    787
Name: count, dtype: int64

In [29]:
test_df['questionType'].value_counts()

questionType
0    670
1    669
Name: count, dtype: int64

## 2.Preprocessing

### Tokenizing

In [30]:
from torchtext.data.utils import get_tokenizer

tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
tokens    = tokenizer("What is the best product?")
tokens

['What', 'is', 'the', 'best', 'product', '?']

### Text to integers (numeral)

In [31]:
from torchtext.vocab import build_vocab_from_iterator

def yield_tokens(data):
    for data_sample in data:
        yield tokenizer(data_sample) 
        
vocab = build_vocab_from_iterator(yield_tokens(train_df['question']), specials = ['<unk>', '<pad>', '<bos>', '<eos>'])
vocab.set_default_index(vocab["<unk>"])

In [32]:
vocab(['here', 'it', 'is'])

[464, 10, 7]

In [33]:
mapping = vocab.get_itos()
mapping[0]

'<unk>'

In [34]:
len(vocab)

10277

## 3. Data loader

### FastText Embedding

In [35]:
from torchtext.vocab import FastText
fast_vectors = FastText(language='simple')

In [36]:
fast_embedding = fast_vectors.get_vecs_by_tokens(vocab.get_itos()).to(device)

In [37]:
fast_embedding.shape

torch.Size([10277, 300])

In [38]:
class DataWrap(Dataset):

    def __init__(self, dataframe):
        self.dataframe = dataframe
    
    def __len__(self):
        return len(self.dataframe)
        
    def __getitem__(self, idx):
        return self.dataframe.iloc[idx]

In [39]:
train_df.iloc[0]

questionType                                                    1
question        i am considering them for my 2014 silver ado d...
Name: 1388, dtype: object

In [40]:
train = DataWrap(train_df)
valid = DataWrap(val_df)
test = DataWrap(test_df)

In [41]:
text_pipeline  = lambda x: vocab(tokenizer(x)) #{hello world this is yt} => {'hello', 'world', 'this', 'is', 'yt'} => {4, 88, 11, 22, 6}

In [42]:
text_pipeline("I am currently teaching LSTM")

[0, 64, 1031, 0, 0]

In [43]:
from torch.utils.data   import DataLoader
from torch.nn.utils.rnn import pad_sequence

pad_idx = vocab['<pad>'] 

def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(_label)
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
    #criterion expects float labels
    return torch.tensor(label_list, dtype=torch.int64), pad_sequence(text_list, padding_value=pad_idx, batch_first=True)

In [44]:
batch_size = 64

train_loader = DataLoader(train, batch_size=batch_size, shuffle=True,  collate_fn=collate_batch) #num_workers to train faster
val_loader   = DataLoader(valid, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
test_loader  = DataLoader(test,  batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [45]:
for label, text in val_loader:
    break

In [46]:
label.shape #(batch_size, )

torch.Size([64])

In [47]:
text.shape #(batch_size, seq len)

torch.Size([64, 47])

## 4. Model (CNN)

In [48]:
class CNN(nn.Module): #more elegant version
    def __init__(self, input_dim, emb_dim, output_dim, dropout, n_filters, filter_sizes):
        
        super().__init__()
                
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, emb_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        embedded = self.embedding(text)
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))
        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

## 5. Train

In [49]:
#explicitly initialize weights for better learning
def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, (nn.Conv2d, nn.Conv2d)):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.kaiming_normal_(param) 

In [50]:
def accuracy(preds, y):
    
    predicted = torch.max(preds.data, 1)[1]
    batch_corr = (predicted == y).sum()
    acc = batch_corr / len(y)
    
    return acc

In [51]:
def train(model, loader, optimizer, criterion, loader_length):
    epoch_loss = 0
    epoch_acc = 0
    model.train() #useful for batchnorm and dropout
    
    for i, (label, text) in enumerate(loader): 
        label = label.to(device) #(batch_size, )
        text = text.to(device) #(batch_size, seq len)
                
        #predict
        predictions = model(text).squeeze(1) #output by the fc is (batch_size, 1), thus need to remove this 1
        
        #calculate loss
        loss = criterion(predictions, label)
        acc  = accuracy(predictions, label)
        
        #backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
                        
    return epoch_loss / loader_length, epoch_acc / loader_length

In [52]:
def evaluate(model, loader, criterion, loader_length):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for i, (label, text) in enumerate(loader): 
            label = label.to(device) #(batch_size, )
            text  = text.to(device)  #(seq len, batch_size)

            predictions = model(text).squeeze(1) 
            
            loss = criterion(predictions, label)
            acc  = accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / loader_length, epoch_acc / loader_length

### Actual training

In [53]:
train_loader_length = len(list(iter(train_loader)))
val_loader_length   = len(list(iter(val_loader)))
test_loader_length  = len(list(iter(test_loader)))

In [54]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [55]:
#experiment tracking
import mlflow
from mlflow.models import infer_signature
import os

# This the dockerized method.
# We build two docker containers, one for python/jupyter and another for mlflow.
# The url `mlflow` is resolved into another container within the same composer.
mlflow.set_tracking_uri("http://mlflow:5000")
# In the dockerized way, the user who runs this code will be `root`.
# The MLflow will also log the run user_id as `root`.
# To change that, we need to set this environ["LOGNAME"] to your name.
os.environ["LOGNAME"] = "noppawee"
#mlflow.create_experiment(name="noppawee-ML-project")  #create if you haven't create
mlflow.set_experiment(experiment_name="CNN500_question_classification")


/usr/local/lib/python3.10/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
2024/04/15 22:10:29 INFO mlflow.tracking.fluent: Experiment with name 'CNN500_question_classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/631739583799255682', creation_time=1713193829157, experiment_id='631739583799255682', last_update_time=1713193829157, lifecycle_stage='active', name='CNN500_question_classification', tags={}>

In [56]:
import time
import torch.optim as optim

num_epochs = [10,20,30]
n_filters = [50,100,150]


for num_epoch in num_epochs:
    for n_filter in n_filters:
        
        input_dim  = len(vocab)
        emb_dim    = 300
        output_dim = 2 #2 classes

        #for cnn
        dropout = 0.5
        filter_sizes = [3, 4, 5]

        params={"model":"CNN", "num_epochs":num_epoch, "n_filters":n_filter, "filter_sizes":filter_sizes, "input_dim":input_dim, "emb_dim":emb_dim, "output_dim":output_dim, "dropout":0.5}
        mlflow.start_run(run_name=f"CNN500-{params['num_epochs']}-epochs-{params['n_filters']}-n_filters")
        mlflow.log_params(params)

        print("="*5, f"CNN with {params['num_epochs']}-epochs-{params['n_filters']}-n_filters","="*5)

        model = CNN(input_dim, emb_dim, output_dim, dropout, n_filter, filter_sizes).to(device)
        model.apply(initialize_weights)
        model.embedding.weight.data = fast_embedding

        lr=0.05
        #training hyperparameters
        optimizer = optim.SGD(model.parameters(), lr=lr)
        criterion = nn.CrossEntropyLoss()

        train_losses, train_accs, val_losses, val_accs = [],[],[],[]
        best_valid_loss = float('inf')

        for epoch in range(num_epoch):
            start_time = time.time()
                
            train_loss, train_acc = train(model, train_loader, optimizer, criterion, train_loader_length)
            valid_loss, valid_acc = evaluate(model, val_loader, criterion, val_loader_length)
                
            #for plotting
            train_losses.append(train_loss)
            train_accs.append(train_acc)
            val_losses.append(valid_loss)
            val_accs.append(valid_acc)
                
            end_time = time.time()
                
            epoch_mins, epoch_secs = epoch_time(start_time, end_time)
            mlflow.log_metric(key="train_loss", value=train_loss, step=epoch)
            mlflow.log_metric(key="train_acc", value=train_acc, step=epoch)
            mlflow.log_metric(key="val_loss", value=valid_loss, step=epoch)
            mlflow.log_metric(key="val_acc", value=valid_acc, step=epoch)
                
            
            #early stopping
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                mlflow.pytorch.log_model(model, "model")
                
            print(f'Epoch: {epoch+1} | Time: {epoch_mins}m {epoch_secs}s')
            print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
            print(f'\tVal.  Loss: {valid_loss:.3f} | Val Acc: {valid_acc*100:.2f}%')
        mlflow.log_metric(key="min_val_loss", value=min(val_losses), step=epoch)    
        mlflow.end_run()
        

===== CNN with 10-epochs-50-n_filters =====
Epoch: 1 | Time: 0m 5s
	Train Loss: 0.694 | Train Acc: 58.67%
	Val.  Loss: 0.592 | Val Acc: 71.43%


/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Epoch: 2 | Time: 0m 5s
	Train Loss: 0.552 | Train Acc: 72.14%
	Val.  Loss: 0.452 | Val Acc: 80.63%
Epoch: 3 | Time: 0m 5s
	Train Loss: 0.446 | Train Acc: 79.07%
	Val.  Loss: 0.396 | Val Acc: 83.26%
Epoch: 4 | Time: 0m 5s
	Train Loss: 0.383 | Train Acc: 83.86%
	Val.  Loss: 0.342 | Val Acc: 85.66%
Epoch: 5 | Time: 0m 5s
	Train Loss: 0.356 | Train Acc: 84.92%
	Val.  Loss: 0.327 | Val Acc: 87.63%
Epoch: 6 | Time: 0m 5s
	Train Loss: 0.333 | Train Acc: 86.34%
	Val.  Loss: 0.309 | Val Acc: 88.03%
Epoch: 7 | Time: 0m 5s
	Train Loss: 0.311 | Train Acc: 87.42%
	Val.  Loss: 0.306 | Val Acc: 87.72%
Epoch: 8 | Time: 0m 5s
	Train Loss: 0.293 | Train Acc: 88.10%
	Val.  Loss: 0.298 | Val Acc: 87.97%
Epoch: 9 | Time: 0m 6s
	Train Loss: 0.275 | Train Acc: 89.07%
	Val.  Loss: 0.283 | Val Acc: 88.59%
Epoch: 10 | Time: 0m 5s
	Train Loss: 0.258 | Train Acc: 89.67%
	Val.  Loss: 0.323 | Val Acc: 87.84%
===== CNN with 10-epochs-100-n_filters =====
Epoch: 1 | Time: 0m 8s
	Train Loss: 0.707 | Train Acc: 57.43%
	